# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 27 2022 9:16AM,238243,10,0085717721,ISDIN Corporation,Released
1,May 27 2022 9:16AM,238243,10,0085717726,ISDIN Corporation,Released
2,May 27 2022 9:16AM,238243,10,0085717727,ISDIN Corporation,Released
3,May 27 2022 9:16AM,238243,10,0085717730,ISDIN Corporation,Released
4,May 27 2022 9:16AM,238243,10,0085717735,ISDIN Corporation,Released
5,May 27 2022 9:16AM,238243,10,0085717739,ISDIN Corporation,Released
6,May 27 2022 9:16AM,238243,10,0085717740,ISDIN Corporation,Released
7,May 27 2022 9:16AM,238243,10,0085717756,ISDIN Corporation,Released
8,May 27 2022 9:16AM,238243,10,0085717757,ISDIN Corporation,Released
9,May 27 2022 9:16AM,238243,10,0085717760,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,238239,Released,2
34,238240,Released,1
35,238241,Released,7
36,238242,Executing,1
37,238243,Released,27


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
238239,NaN,2.0
238240,NaN,1.0
238241,NaN,7.0
238242,1.0,NaN
238243,NaN,27.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
238153,1.0,0.0
238159,0.0,1.0
238163,0.0,1.0
238165,1.0,8.0
238168,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
238153,1,0
238159,0,1
238163,0,1
238165,1,8
238168,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,238153,1,0
1,238159,0,1
2,238163,0,1
3,238165,1,8
4,238168,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,238153,1,
1,238159,,1
2,238163,,1
3,238165,1,8
4,238168,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 27 2022 9:16AM,238243,10,ISDIN Corporation
27,May 27 2022 9:09AM,238242,16,"SHL Pharma, LLC"
28,May 27 2022 8:54AM,238241,15,"Virtus Pharmaceuticals, LLC"
35,May 27 2022 8:49AM,238240,10,"Senseonics, Incorporated"
36,May 27 2022 8:48AM,238239,15,"Carwin Pharmaceutical Associates, LLC"
38,May 27 2022 8:47AM,238238,15,"Alliance Pharma, Inc."
40,May 27 2022 8:46AM,238237,10,Beach Products Inc
55,May 27 2022 8:46AM,238234,15,"Person & Covey, Inc."
60,May 27 2022 8:43AM,238236,10,Emerginnova
61,May 27 2022 8:32AM,238235,10,"E5 Pharma, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,May 27 2022 9:16AM,238243,10,ISDIN Corporation,,27
1,May 27 2022 9:09AM,238242,16,"SHL Pharma, LLC",1,
2,May 27 2022 8:54AM,238241,15,"Virtus Pharmaceuticals, LLC",,7
3,May 27 2022 8:49AM,238240,10,"Senseonics, Incorporated",,1
4,May 27 2022 8:48AM,238239,15,"Carwin Pharmaceutical Associates, LLC",,2
5,May 27 2022 8:47AM,238238,15,"Alliance Pharma, Inc.",,2
6,May 27 2022 8:46AM,238237,10,Beach Products Inc,,15
7,May 27 2022 8:46AM,238234,15,"Person & Covey, Inc.",,5
8,May 27 2022 8:43AM,238236,10,Emerginnova,,1
9,May 27 2022 8:32AM,238235,10,"E5 Pharma, LLC",,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,May 27 2022 9:16AM,238243,10,ISDIN Corporation,27,
1,May 27 2022 9:09AM,238242,16,"SHL Pharma, LLC",,1
2,May 27 2022 8:54AM,238241,15,"Virtus Pharmaceuticals, LLC",7,
3,May 27 2022 8:49AM,238240,10,"Senseonics, Incorporated",1,
4,May 27 2022 8:48AM,238239,15,"Carwin Pharmaceutical Associates, LLC",2,
5,May 27 2022 8:47AM,238238,15,"Alliance Pharma, Inc.",2,
6,May 27 2022 8:46AM,238237,10,Beach Products Inc,15,
7,May 27 2022 8:46AM,238234,15,"Person & Covey, Inc.",5,
8,May 27 2022 8:43AM,238236,10,Emerginnova,1,
9,May 27 2022 8:32AM,238235,10,"E5 Pharma, LLC",2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,May 27 2022 9:16AM,238243,10,ISDIN Corporation,27,
1,May 27 2022 9:09AM,238242,16,"SHL Pharma, LLC",,1
2,May 27 2022 8:54AM,238241,15,"Virtus Pharmaceuticals, LLC",7,
3,May 27 2022 8:49AM,238240,10,"Senseonics, Incorporated",1,
4,May 27 2022 8:48AM,238239,15,"Carwin Pharmaceutical Associates, LLC",2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,May 27 2022 9:16AM,238243,10,ISDIN Corporation,27.0,NaN
1,May 27 2022 9:09AM,238242,16,"SHL Pharma, LLC",NaN,1.0
2,May 27 2022 8:54AM,238241,15,"Virtus Pharmaceuticals, LLC",7.0,NaN
3,May 27 2022 8:49AM,238240,10,"Senseonics, Incorporated",1.0,NaN
4,May 27 2022 8:48AM,238239,15,"Carwin Pharmaceutical Associates, LLC",2.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,May 27 2022 9:16AM,238243,10,ISDIN Corporation,27.0,0.0
1,May 27 2022 9:09AM,238242,16,"SHL Pharma, LLC",0.0,1.0
2,May 27 2022 8:54AM,238241,15,"Virtus Pharmaceuticals, LLC",7.0,0.0
3,May 27 2022 8:49AM,238240,10,"Senseonics, Incorporated",1.0,0.0
4,May 27 2022 8:48AM,238239,15,"Carwin Pharmaceutical Associates, LLC",2.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2858506,78.0,0.0
12,238210,1.0,0.0
15,1191139,42.0,0.0
16,1429245,13.0,1.0
18,238159,1.0,0.0
19,952795,4.0,21.0
20,952687,16.0,3.0
21,238163,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2858506,78.0,0.0
1,12,238210,1.0,0.0
2,15,1191139,42.0,0.0
3,16,1429245,13.0,1.0
4,18,238159,1.0,0.0
5,19,952795,4.0,21.0
6,20,952687,16.0,3.0
7,21,238163,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,78.0,0.0
1,12,1.0,0.0
2,15,42.0,0.0
3,16,13.0,1.0
4,18,1.0,0.0
5,19,4.0,21.0
6,20,16.0,3.0
7,21,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,78.0
1,12,Released,1.0
2,15,Released,42.0
3,16,Released,13.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21
Status,,,,,,,,
Executing,0.0,0.0,0.0,1.0,0.0,21.0,3.0,0.0
Released,78.0,1.0,42.0,13.0,1.0,4.0,16.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21
0,Executing,0.0,0.0,0.0,1.0,0.0,21.0,3.0,0.0
1,Released,78.0,1.0,42.0,13.0,1.0,4.0,16.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21
0,Executing,0.0,0.0,0.0,1.0,0.0,21.0,3.0,0.0
1,Released,78.0,1.0,42.0,13.0,1.0,4.0,16.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()